##Connect Spark 3.2.2

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://cloud.r-project.org/bin/li

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-19 19:09:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-07-19 19:09:11 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("final_project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

##Load csv from S3 to Colab

In [8]:
from pyspark import SparkFiles
url = "https://video-game-dataset-uot-boot-camp-2022-group-4.s3.us-east-2.amazonaws.com/sales_cleaned.csv"
spark.sparkContext.addFile(url)
cleaned_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("sales_cleaned.csv"), sep=",", header=True, inferSchema=True)
cleaned_df.show()

+----+--------------------+------------+--------+--------------------+--------------------+------+-----------+
|Rank|                Name|       Genre|Platform|           Publisher|         Developer_x|  Year|Total_Sales|
+----+--------------------+------------+--------+--------------------+--------------------+------+-----------+
|   1|          Wii Sports|      Sports|     Wii|            Nintendo|        Nintendo EAD|2006.0|      82.86|
|   2|   Super Mario Bros.|    Platform|     NES|            Nintendo|        Nintendo EAD|1985.0|      40.24|
|   3|      Mario Kart Wii|      Racing|     Wii|            Nintendo|        Nintendo EAD|2008.0|      37.14|
|   4|PlayerUnknown's B...|     Shooter|      PC|    PUBG Corporation|    PUBG Corporation|2017.0|       36.6|
|   5|   Wii Sports Resort|      Sports|     Wii|            Nintendo|        Nintendo EAD|2009.0|      33.09|
|   6|Pokemon Red / Gre...|Role-Playing|      GB|            Nintendo|          Game Freak|1998.0|      31.38|
|

In [9]:
from pyspark import SparkFiles
url = "https://video-game-dataset-uot-boot-camp-2022-group-4.s3.us-east-2.amazonaws.com/all_columns_df.csv"
spark.sparkContext.addFile(url)
all_columns_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("all_columns_df.csv"), sep=",", header=True, inferSchema=True)
all_columns_df.show()

+----+--------------------+------------+-----------+--------+--------------------+--------------------+------------+----------+------+-------------+-----------+
|Rank|                Name|       Genre|ESRB_Rating|Platform|           Publisher|         Developer_x|Critic_Score|User_Score|  Year|      Country|Total_Sales|
+----+--------------------+------------+-----------+--------+--------------------+--------------------+------------+----------+------+-------------+-----------+
|   1|          Wii Sports|      Sports|          E|     Wii|            Nintendo|        Nintendo EAD|         7.7|      null|2006.0|        Japan|      82.86|
|   2|   Super Mario Bros.|    Platform|       null|     NES|            Nintendo|        Nintendo EAD|        10.0|      null|1985.0|        Japan|      40.24|
|   3|      Mario Kart Wii|      Racing|          E|     Wii|            Nintendo|        Nintendo EAD|         8.2|       9.1|2008.0|        Japan|      37.14|
|   4|PlayerUnknown's B...|     Sh

##Connect RDS to PostgreSQL table

In [ ]:
# Configure settings for RDS
# "password": "*********", 
from getpass import getpass
password = getpass('password')
mode = "append"
jdbc_url="jdbc:postgresql://projectdb.cqnk8h8kczwj.us-east-2.rds.amazonaws.com:5432/project_db"
config = {"user":"postgresRDS", 
          "password": password,
          "driver":"org.postgresql.Driver"}

##Write dataframe to table

In [ ]:
# Write cleaned_sales to table in RDS
cleaned_df.write.jdbc(url=jdbc_url, table='cleaned_df', mode=mode, properties=config)

In [ ]:
# Write all_columns_df to table in RDS
all_columns_df.write.jdbc(url=jdbc_url, table='all_columns_df', mode=mode, properties=config)